In [ ]:
# imports
import os
import pandas as pd
import random
from datasets import Dataset, DatasetDict
import torch
import numpy as np

import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BatchEncoding, T5Tokenizer, T5ForConditionalGeneration, MT5Tokenizer, MT5ForConditionalGeneration
from datasets import load_from_disk
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from transformers import BertForSequenceClassification
from sklearn.preprocessing import LabelEncoder

from torch.optim import AdamW
from transformers import get_scheduler
import evaluate
from torch.optim.lr_scheduler import ChainedScheduler, ExponentialLR

from tqdm.auto import tqdm
from google.colab import files

In [ ]:
%%capture
!pip install datasets
!pip install evaluate

In [ ]:
# Connet to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Create_augmentations.py

In [ ]:
import os.path
import pandas as pd
import random
from datasets import Dataset, DatasetDict
import torch
import numpy as np

def is_running_in_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False


data_dir = 'datasets/'
if is_running_in_colab():
    data_dir = "drive/MyDrive/HHSC/"


def random_replace(string, default_prob):
    drop_prob = default_prob / 6
    space_prob = default_prob / 6
    replacements = {
        'א': [('ע', default_prob), ('ה', default_prob)],
        'ע': [('א', default_prob), ('ה', default_prob)],
        'ה': [('א', default_prob), ('ע', default_prob)],

        'ט': [('ת', default_prob)],
        'ת': [('ט', default_prob)],

        'ח': [('כ', default_prob)],
        'כ': [('ח', default_prob), ('ק', default_prob)],
        'ק': [('כ', default_prob)],

        'ש': [('ס', default_prob / 2)],
        'ס': [('ש', default_prob / 2)],

        'ב': [('ו', default_prob / 4)],
        'ו': [('ב', default_prob / 4)],

        'ג': [('ז', default_prob / 4)],
        'ז': [('ג', default_prob / 4)]
    }

    # Convert string to list to make replacements
    string_list = list(string)
    previous_was_changed = False  # we avoid dropping a few letters in a row
    for idx, char in enumerate(string_list):
        if char in replacements:
            for replacement, prob in replacements[char]:
                if random.random() < prob:  # Unique probability for each replacement
                    string_list[idx] = replacement
                    previous_was_changed = True
                    break  # Stop after the first replacement
        if previous_was_changed is False and (random.random() < drop_prob):  # Randomly decide to drop a letter
            string_list[idx] = ''

    new_string_list = []
    previous_was_dropped = False  # we avoid dropping a few letters in a row
    for char in string_list:
        new_string_list.append(char)
        if (previous_was_dropped is False) and (random.random() < space_prob):  # Randomly decide to add a space
            new_string_list.append(' ')
            previous_was_dropped = True
            continue
        previous_was_dropped = False

    return ''.join(new_string_list)


def create_augmentations(percentage=20, verbose=False, truncate_min=1, truncate_max=13):
    default_prob = float(percentage) / 100

    input_txt_path = data_dir + 'hebrew_text.txt'
    output_path = data_dir + 'hebrew_text_aug_' + str(percentage)

    # Read the input TXT file
    with open(input_txt_path, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()

    # Process each line
    processed_lines = [lines[0]]
    for long_line in lines[1:]:
        long_line = long_line.strip()
        if truncate_min is None or truncate_max is None:
            current_lines = [long_line]
        else:
            words = long_line.split()
            current_lines = []
            i = 0
            while i < len(words):
                num_words = np.random.randint(truncate_min, truncate_max)
                current_lines.append(' '.join(words[i:i+num_words]))
                i += num_words
        for line in current_lines:
            modified_line = random_replace(line, default_prob)
            if line == '' or len(line) < 2:
                continue
            processed_lines.append(f"{line}\t{modified_line}")

    if verbose:
        print(f'-----------> Example:\n\n')
        print(processed_lines[1])
        print(f'<-----------= Example:\n\n')

    # Save data in txt format - uncomment to activate
    # # Write the original and modified text to the output TXT file
    # output_txt_path = output_path + '.txt'
    # with open(output_txt_path, 'w', encoding='utf-8') as outfile:
    #     outfile.write('\n'.join(processed_lines))
    #
    # print(f"Modified data saved to {output_txt_path}")

    print(f'Exporting the data to Excel file...')
    print(f'{len(processed_lines)-1} lines.')

    processed_lines = processed_lines[1:]
    data = [line.strip().split('\t') for line in processed_lines]
    df = pd.DataFrame(data, columns=['original', 'errors'])  # Adjust column names as needed
    excel_output_path = output_path + '.xlsx'
    df.to_excel(excel_output_path, index=False, engine='openpyxl')

    print(f"Conversion complete. Check {excel_output_path}")
    return excel_output_path


def export_dataset(excel_path):
    df = pd.read_excel(excel_path)
    df.dropna(subset=['errors', 'original'], inplace=True)
    texts_with_errors = df['errors'].tolist()
    texts_corrected = df['original'].tolist()

    data_dict = {
        'errors': texts_with_errors,
        'original': texts_corrected
    }

    # dataset = ds.Dataset.from_dict(data_dict)
    dataset = Dataset.from_dict(data_dict)

    return dataset

def create_or_load_dataset(augemntation_percentage=30, test_size=0.2, verbose=False):
    train_path = data_dir + 'train.pt'
    test_path = data_dir + 'test.pt'
    if (not os.path.exists(train_path)) or (not os.path.exists(test_path)):
        print('Creating & Saving Dataset...')
        excel_path = create_augmentations(augemntation_percentage, verbose)
        dataset = export_dataset(excel_path)
        # Split the dataset into training and testing sets
        train_test_split = dataset.train_test_split(test_size=test_size)
        torch.save(train_test_split['train'], train_path)
        torch.save(train_test_split['test'], test_path)
        return train_test_split['train'], train_test_split['test']
    else:
        print('Loading Dataset from file...')
        train_split = torch.load(train_path)
        test_split = torch.load(test_path)
        return train_split, test_split

def export_train_test_dataset(excel_path, test_size=0.2):
    train_path = data_dir + 'train.pt'
    test_path = data_dir + 'test.pt'
    if (not os.path.exists(train_path)) and (not os.path.exists(test_path)):
        print('Creating & Saving Dataset...')
        dataset = export_dataset(excel_path)
        # Split the dataset into training and testing sets
        train_test_split = dataset.train_test_split(test_size=test_size)
        torch.save(train_test_split['train'], train_path)
        torch.save(train_test_split['test'], test_path)

        return train_test_split['train'], train_test_split['test']
    else:
        print('Loading Dataset from file...')
        train_split = torch.load(train_path)
        test_split = torch.load(test_path)
        return train_split, test_split

def full_run(percentage=30, verbose=False):
    return export_dataset(create_augmentations(percentage, verbose))


def full_run_train_test_split(percentage=30, verbose=True):
    return export_train_test_dataset(create_augmentations(percentage, verbose))


#full_run_train_test_split()

# transformer_prepare_data.py

In [ ]:
import pandas as pd
from datasets import Dataset
# import tensorflow as tf
# from create_augmentations import *
from transformers import BertTokenizer, BatchEncoding, T5Tokenizer, T5ForConditionalGeneration, MT5Tokenizer, MT5ForConditionalGeneration
from datasets import load_from_disk
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from transformers import BertForSequenceClassification
from sklearn.preprocessing import LabelEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

X_NAME = 'errors'  # Todo: change names
Y_NAME = 'original'

# ---------- HYPERPARAMETERS -----------
# -------------------------------------->
max_length = 128
# <--------------------------------------


# --------- HELPER FUNCTIONS -----------
# -------------------------------------->
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items() if key in ['input_ids', 'attention_mask']}
        # Ensure labels are correctly indexed
        if isinstance(self.labels, BatchEncoding):
            item['labels'] = self.labels['input_ids'][idx]  # Adjust according to how labels are stored
        else:
            item['labels'] = self.labels[idx].clone().detach()

        return item

    def __len__(self):
        # return len(self.labels)
        return len(self.encodings['input_ids'])


class Seq2SeqDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer, max_length=128):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        print(f"Index: {idx}, Type: {type(idx)}")
        if isinstance(idx, list):
            raise ValueError("Index must be an integer, not a list")

        input_text = self.inputs[idx]
        target_text = self.targets[idx]

        input_encoding = self.tokenizer(
            input_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        target_encoding = self.tokenizer(
            target_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encoding['input_ids'].squeeze(),
            'attention_mask': input_encoding['attention_mask'].squeeze(),
            'labels': target_encoding['input_ids'].squeeze()
        }

# <--------------------------------------


def get_model():
    # model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
    # tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
    # model = T5ForConditionalGeneration.from_pretrained('t5-small')
    # tokenizer = T5Tokenizer.from_pretrained('t5-small')

    model_name = "google/mt5-small"
    tokenizer = MT5Tokenizer.from_pretrained(model_name)
    model = MT5ForConditionalGeneration.from_pretrained(model_name)


    # --------- FREEZING LAYERS ------------
    # -------------------------------------->

    # for name, param in model.named_parameters():
    #     if name.startswith("encoder.block.2."):
    #         param.requires_grad = False

    # for name, param in model.named_parameters():
    #     if name.startswith("encoder.block.3."):
    #         param.requires_grad = False

    # for name, param in model.named_parameters():
    #     if name.startswith("encoder.block.4."):
    #         param.requires_grad = False

    for name, param in model.named_parameters():
        if name.startswith("encoder.block.5."):
            param.requires_grad = False

    for name, param in model.named_parameters():
        if name.startswith("encoder.block.6."):
            param.requires_grad = False

    for name, param in model.named_parameters():
        if name.startswith("encoder.block.7."):
            param.requires_grad = False

    for name, param in model.named_parameters():
        if name.startswith("decoder.block.0."):
            param.requires_grad = False

    for name, param in model.named_parameters():
        if name.startswith("decoder.block.1."):
            param.requires_grad = False

    for name, param in model.named_parameters():
        if name.startswith("decoder.block.2."):
            param.requires_grad = False

    for name, param in model.named_parameters():
        if name.startswith("decoder.block.3."):
            param.requires_grad = False

    for name, param in model.named_parameters():
        if name.startswith("decoder.block.4."):
            param.requires_grad = False

    for name, param in model.named_parameters():
        if name.startswith("decoder.block.5."):
            param.requires_grad = False

    # for name, param in model.named_parameters():
    #     if name.startswith("decoder.block.6."):
    #         param.requires_grad = False

    # <--------------------------------------

    # ----------SEEING THE MODEL------------
    # -------------------------------------->
    print()
    print('--- Printing the layers of the model ---')
    for name, param in model.named_parameters():
        print(name, param.requires_grad)
    print('--- Done printing the layers of the model ---')
    print()

    # <--------------------------------------
    return model, tokenizer


def prepare_data(tokenizer, num_train=None, num_test=None):
    # -------------- DATASET ---------------
    # -------------------------------------->
    dataset_train, dataset_test = create_or_load_dataset(verbose=False)
    dataset_train.set_format('pytorch')
    dataset_test.set_format('pytorch')
    train_inputs = dataset_train[X_NAME]
    train_labels = dataset_train[Y_NAME]
    test_inputs = dataset_test[X_NAME]
    test_labels = dataset_test[Y_NAME]

    if num_train is not None and num_train > 0:
      train_inputs = train_inputs[:num_train]
      train_labels = train_labels[:num_train]
    if num_test is not None and num_test > 0:
      test_inputs = test_inputs[:num_test]
      test_labels = test_labels[:num_test]
    print(f'Number of training samples = {len(train_labels)}')
    print(f'Number of test samples = {len(test_labels)}')


    def truncate_sentences(sentences, sentences_target):
        truncated_sentences = []
        truncated_sentences_targets = []
        for sentence, label in zip(sentences, sentences_target):
            words = sentence.split()
            labels = label.split()
            num_words = random.randint(1, 13)  # Random number between 1 and 13
            truncated_sentence = ' '.join(words[:num_words])
            truncated_sentence_target = ' '.join(labels[:num_words])
            truncated_sentences.append(truncated_sentence)
            truncated_sentences_targets.append(truncated_sentence_target)
        print('----------------------')
        print('Data after truncation:')
        print(f'truncated input:\n{truncated_sentences[1]}')
        print(f'truncated label:\n{truncated_sentences_targets[1]}')
        print('----------------------')
        return truncated_sentences, truncated_sentences_targets

    #train_inputs, train_labels = truncate_sentences(train_inputs, train_labels)
    #test_inputs, test_labels = truncate_sentences(test_inputs, test_labels)

    train_input_tokenized = tokenizer(train_inputs, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
    train_labels_tokenized = tokenizer(train_labels, truncation=True, padding=True, max_length=max_length, return_tensors='pt').input_ids
    test_input_tokenized = tokenizer(test_inputs, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
    test_labels_tokenized = tokenizer(test_labels, truncation=True, padding=True, max_length=max_length, return_tensors='pt').input_ids

    text_tensor_train_ds = TextDataset(train_input_tokenized, train_labels_tokenized)
    text_tensor_test_ds = TextDataset(test_input_tokenized, test_labels_tokenized)

    # text_tensor_train_ds = Seq2SeqDataset(train_input_tokenized, train_labels_tokenized, tokenizer, max_length=128)
    # text_tensor_test_ds = Seq2SeqDataset(test_input_tokenized, test_labels_tokenized, tokenizer, max_length=128)

    # Save the datasets to disk
    # torch.save(text_tensor_train_ds, 'tokenized/text_tensor_train_ds.pt')
    # torch.save(text_tensor_test_ds, 'tokenized/text_tensor_test_ds.pt')
    return text_tensor_train_ds, text_tensor_test_ds


def get_model_and_data(path_to_data='tokenized', num_train=None, num_test=None):
    model, tokenizer = get_model()
    # prepare_data(tokenizer)  # todo: remove this line
    text_tensor_train_ds, text_tensor_test_ds = prepare_data(tokenizer, num_train, num_test)

    return model, tokenizer, text_tensor_train_ds, text_tensor_test_ds




# transformer.py

In [ ]:
# import pandas as pd
# from datasets import Dataset
# from create_augmentations import *
# from datasets import load_from_disk
# import os
# import torch.nn as nn
# from torch.utils.data import DataLoader, TensorDataset
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from transformers import BertForSequenceClassification
# from sklearn.preprocessing import LabelEncoder
# from transformers import BertTokenizer, TrainingArguments, Trainer, BatchEncoding, TrainerCallback
# from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
# import torch
# from transformer_prepare_data import *
from torch.optim import AdamW
from transformers import get_scheduler
import evaluate
from torch.optim.lr_scheduler import ChainedScheduler, ExponentialLR


# ---------- HYPERPARAMETERS -----------
# -------------------------------------->
BATCH_SIZE = 16
num_epochs = 15
num_train = None
num_test = 1000
lr = 1e-3
accumulation_steps = 4
# <--------------------------------------


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Load the saved datasets
model, tokenizer, text_tensor_train_ds, text_tensor_test_ds = get_model_and_data(num_train=num_train, num_test=num_test)
model.to(device)
# text_tensor_train_ds = torch.load('tokenized/text_tensor_train_ds.pt')
# text_tensor_test_ds = torch.load('tokenized/text_tensor_test_ds.pt')


def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}


train_dataloader = DataLoader(text_tensor_train_ds, shuffle=True, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(text_tensor_test_ds, batch_size=BATCH_SIZE)

num_training_steps = num_epochs * len(train_dataloader)
from transformers import Adafactor
optimizer = Adafactor(model.parameters(), scale_parameter=False, relative_step=False, warmup_init=False, lr=lr)
from transformers import get_cosine_schedule_with_warmup
cosine_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=100,
    num_training_steps=num_training_steps
)
exp_scheduler = ExponentialLR(optimizer, gamma=0.9)
lr_scheduler = ChainedScheduler([cosine_scheduler, exp_scheduler])
scaler = torch.cuda.amp.GradScaler()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]


--- Printing the layers of the model ---
shared.weight True
encoder.block.0.layer.0.SelfAttention.q.weight True
encoder.block.0.layer.0.SelfAttention.k.weight True
encoder.block.0.layer.0.SelfAttention.v.weight True
encoder.block.0.layer.0.SelfAttention.o.weight True
encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight True
encoder.block.0.layer.0.layer_norm.weight True
encoder.block.0.layer.1.DenseReluDense.wi_0.weight True
encoder.block.0.layer.1.DenseReluDense.wi_1.weight True
encoder.block.0.layer.1.DenseReluDense.wo.weight True
encoder.block.0.layer.1.layer_norm.weight True
encoder.block.1.layer.0.SelfAttention.q.weight True
encoder.block.1.layer.0.SelfAttention.k.weight True
encoder.block.1.layer.0.SelfAttention.v.weight True
encoder.block.1.layer.0.SelfAttention.o.weight True
encoder.block.1.layer.0.layer_norm.weight True
encoder.block.1.layer.1.DenseReluDense.wi_0.weight True
encoder.block.1.layer.1.DenseReluDense.wi_1.weight True
encoder.block.1.layer.1.DenseR

In [ ]:
import numpy as np
from tqdm.auto import tqdm
from google.colab import files
progress_bar = tqdm(range(num_training_steps))

def calc_accuracy(model, tokenizer, test_dataloader):
    metric = evaluate.load("accuracy")
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    total_examples = 0

    with torch.no_grad():  # Disable gradient calculation for efficiency
        for batch in test_dataloader:
            batch = {key: value.to(device) for key, value in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.item()

            # Generate predictions
            generated_tokens = model.generate(batch['input_ids'], max_length=batch['labels'].shape[1])

            # Decode the generated tokens and labels to text
            #generated_texts = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            #labels_texts = tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)

            # Calculate accuracy by comparing generated texts to target texts
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            predictions = predictions.view(-1)
            references = batch["labels"].view(-1)
            mask = references != tokenizer.pad_token_id
            predictions = predictions[mask]
            references = references[mask]

            metric.add_batch(predictions=predictions, references=references)

    avg_test_loss = total_loss / len(test_dataloader)
    accuracy_dict = metric.compute()
    accuracy = float(accuracy_dict['accuracy'])
    random_index = random.randint(0, len(batch['labels']) - 1)
    random_target = tokenizer.decode(batch['labels'][random_index], skip_special_tokens=True)
    random_prediction = tokenizer.decode(generated_tokens[random_index], skip_special_tokens=True)
    random_current = tokenizer.decode(batch['input_ids'][random_index], skip_special_tokens=True)
    #print(f'Current state:\nTarget: {labels_texts[random_index]}\nPrediction: {generated_texts[random_index]}')
    print(f'Input: {random_current}\nTarget: {random_target}\nPrediction: {random_prediction}')

    model.train()  # Set the model back to training mode
    return avg_test_loss, accuracy

from google.colab import drive
drive.mount('/content/drive')

if os.path.exists('/content/drive/My Drive/saved_checkpoint.pth'):
  checkpoint = torch.load('/content/drive/My Drive/saved_checkpoint.pth')
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  scaler.load_state_dict(checkpoint['scaler_state_dict'])
  lr_scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
  start_epoch = checkpoint['epoch'] + 1
  best_accuracy = checkpoint['best_accuracy']
  print('Model loaded from Google Drive')
else:
  start_epoch = 0
  best_accuracy = 0

model.train()
train_losses = []
test_losses = []
test_accuracies = []

# Training loop
for epoch in range(start_epoch):
  for batch in train_dataloader:
        progress_bar.update(1)
for epoch in range(start_epoch, num_epochs):
    epoch_loss = 0
    for batch_idx, batch in enumerate(train_dataloader):
        batch = {key: value.to(device) for key, value in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Update weights and reset gradients every 'accumulation_steps'
        if (batch_idx + 1) % accumulation_steps == 0:
          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()

        progress_bar.update(1)

        epoch_loss += loss.item()  # Accumulate the loss

    # Calculate average loss for the epoch
    avg_train_loss = epoch_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)
    print(f"\nEpoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}")

    # -------------------------------------------
    model.eval()
    avg_test_loss, accuracy = calc_accuracy(model, tokenizer, test_dataloader)
    test_losses.append(avg_test_loss)
    test_accuracies.append(accuracy)
    print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.4f}  learning reate: {optimizer.param_groups[0]['lr']}")
    # Set the model back to training mode
    model.train()
    # -------------------------------------------

    # Saving checkout
    if accuracy > best_accuracy:
      best_accuracy = accuracy
      #torch.save(model.state_dict(), 'saved_model_checkout.pth')
      # Save the model to your Google Drive
      torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scaler_state_dict': scaler.state_dict(),  # if using mixed precision
            'scheduler_state_dict': lr_scheduler.state_dict(),
            'best_accuracy': best_accuracy,
      }, '/content/drive/My Drive/saved_checkpoint.pth')




torch.save(model.state_dict(), 'saved_model.pth')



  0%|          | 0/197685 [00:00<?, ?it/s]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model loaded from Google Drive


In [ ]:
checkpoint = torch.load('/content/drive/My Drive/saved_checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model_name = "google/mt5-small"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model.eval()
label = 'שגיאות כתיב'
input = 'שגיעוט קטיב'
input_encoded = tokenizer.encode(input)
label_encoded = tokenizer.encode(label)


In [ ]:
input_chen = tokenizer('שגיעוט קטיב', truncation=True, padding=True, max_length=max_length, return_tensors='pt')
input_chen = {'input_ids': input_chen['input_ids'].to(device), 'attention_mask': input_chen['attention_mask'].to(device)}
label_chen = tokenizer('שגיאות כתיב', truncation=True, padding=True, max_length=max_length, return_tensors='pt')
model.eval()
res = model(**input_chen)

ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds

In [ ]:
# Clean GPU catche
del model
del optimizer
torch.cuda.empty_cache()

In [ ]:
import matplotlib.pyplot as plt

num_testing_steps = len(test_dataloader)
progress_bar_test = tqdm(range(num_testing_steps))


metric = evaluate.load("accuracy")
model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    # metric.add_batch(predictions=predictions, references=batch["labels"])

    # Flatten predictions and references
    predictions = predictions.view(-1)
    references = batch["labels"].view(-1)

    # Filter out padding tokens (if applicable)
    mask = references != tokenizer.pad_token_id
    predictions = predictions[mask]
    references = references[mask]

    metric.add_batch(predictions=predictions, references=references)
    progress_bar_test.update(1)


# Compute the final accuracy
final_score = metric.compute()
print("\nAccuracy:", final_score)


# Plotting the loss and accuracy
plt.figure(figsize=(12, 5))

# Plot training and test loss
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs + 1), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# Plot test accuracy
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

plt.tight_layout()
plt.show()

# Checking the trained model
### check_model.py

In [ ]:
import torch
# from transformers import T5Tokenizer, T5ForConditionalGeneration
# from transformer_prepare_data import *

# model, tokenizer = get_model_and_data()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Sample data for inference
new_data = ["זה למשל דוגמא אם שגיעה", "עוד טקסט לטיקון", "הנה טאות", "לא הני"]
new_data_tokenized = tokenizer(
    new_data, max_length=128, padding='max_length', truncation=True, return_tensors='pt'
)

# Move data to device
input_ids = new_data_tokenized['input_ids'].to(device)
attention_mask = new_data_tokenized['attention_mask'].to(device)
# print("Tokenized Input IDs:", input_ids)
# print("Attention Mask:", attention_mask)

# model.load_state_dict(torch.load('spellcheck_model/saved_model.pth'))
model.eval()

with torch.no_grad():
    generated_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=128,
        num_beams=4,
        early_stopping=True
    )


predictions = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]
print("Predictions:")
for i, pred in enumerate(predictions):
    print(f"Input: {new_data[i]}")
    print(f"Output: {pred}")
    print()



Predictions:
Input: זה למשל דוגמא אם שגיעה
Output: זה למשל דוגמא עם שגיעה

Input: עוד טקסט לטיקון
Output: עוד טקסט לתיקון

Input: הנה טאות
Output: הנה טעות

Input: לא הני
Output: לא אני



# Sanity check that the used tokenizer works

In [ ]:
# predictions = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]
input = "שגיאה"
input_encoded = tokenizer.encode(input)
input_decoded = tokenizer.decode(input_encoded, skip_special_tokens=True)

print(f'input: {input}')
print(f'input_encoded: {input_encoded}')
print(f'input_decoded: {input_decoded}')

input: שגיאה
input_encoded: [83407, 49491, 1]
input_decoded: שגיאה


In [ ]:
from google.colab import files

# files.download('saved_model_checkout.pth')
files.download('saved_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# # Save the model to your Google Drive
# torch.save(model.state_dict(), '/content/drive/My Drive/saved_model.pth')
